In [1]:
%matplotlib
#%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re
import pandas as pd
import matplotlib.pyplot as plt
import math

pd.options.mode.use_inf_as_na = True

Using matplotlib backend: TkAgg


In [228]:
filePath = r'\\ait-pdfs.win.dtu.dk\users\homedir\taba\Documents\Experiment design\Danish sentences\europarl-da-sentiment-master'
fileIn = 'europarl-da-sentiment.csv'
fileOut = 'europarl_reduced.txt'

sentencesFinal = list()

# access lines in the file in form of list
try:
    fSentences = open(filePath+'\\'+fileIn, encoding="utf-8")
    sentences = fSentences.readlines()

except:
    print('error opening file')

# add " if not present
line_ind = -1
for line in sentences:
    line_ind = line_ind + 1
    
    # add quote to beginning of sentence
    check_quote = line.find(',"')
    
    if check_quote < 0:
        if re.search('[a-zA-Z]', line) is not None:
            # find position of first comma
            comma_first = line.index(',')
            comma_second = line[comma_first+1:].index(',')
            sentences[line_ind] = line[0:comma_second+comma_first+2] + '"' + line[comma_second+comma_first+2:]
            line = sentences[line_ind]
            
    # add quote to end of sentence
    check_quote_end = line.find('.,,')
    
    if check_quote_end > 0:
        if re.search('[a-zA-Z]', line) is not None:
            sentences[line_ind] = sentences[line_ind].replace('.,,', '.",,')
            line = sentences[line_ind]
        
    else:
        check_exclamation = line.find('!,,')
        if check_exclamation > 0:
            sentences[line_ind] = sentences[line_ind].replace('!,,', '!",,')
            line = sentences[line_ind]

re_extractSentence = re.compile('(?<=")(.*)(?=")')

nCharList = list()
line_ind = -1
for line in sentences:
    line_ind = line_ind + 1
    sentence = re_extractSentence.findall(line)
    if sentence:
            
        # replace 'hr.' with nothing
        sentence = sentence[0].replace('hr .', '')
            
        # replace 'fru.' with nothing
        sentence = sentence.replace('fru', '')
            
        # replace quotation marks in the sentence
        sentence = sentence.replace('"', '')
            
        # replace any brackets
        # check if complete brackets present first
        sentence = sentence.replace('(', '')
        sentence = sentence.replace(')', '')
            
        # replace extra spaces before '
        sentence = sentence.replace(' \'', '\'')
            
        # replace ; with ,
        sentence = sentence.replace(';', ',')
            
        # replace - with ,
        sentence = sentence.replace(' -', ',')
            
        # third last -- replace comma and space with only comma
        sentence = sentence.replace(' ,', ',')
            
        # second last -- replace comma and point with only point
        sentence = sentence.replace(' .', '.')
            
        # in the end -- replace double space marks with single space
        sentence = sentence.replace('  ', ' ')

        # remove sentences with shortforms and numbers 

        if sentence[:-1].find('.') < 0:
                    
            # remove sentences with numbers:
            if not any(char.isdigit() for char in sentence):
                
                if sentence.find('?') < 0:
                        
                    # remove sentence if there is still - in it
                    if sentence.find('-') < 0:
                            
                        # remove sentence if it contains ' without an s before
                        if sentence.find('s\'') < 0:
                            
                            # remove sentences any upper case letters (names):
                            if not any(x.isupper() for x in sentence[1:]):
                                sentence = sentence.lower()
                                
                                # remove the last period
                                sentence = sentence.replace('.', '')
                                    
                                    
                                # remove sentence if it is more than 35 words long (occupies multiple lines in notepad)
                                if len(sentence.split()) < 35:
                                        
                                    # remove sentences if they contain u'é' 
                                    if sentence.find(u'é') < 0:
                                            
                                        if sentence.find(':') < 0:
                                            
                                            # remove sentences that end in ,
                                            if not sentence.endswith(', '):
                                            
                                                # remove small phrases like 'tak.'
                                                if len(sentence) > 20:
                                            
                                                    sentencesFinal.append(sentence)
                                    
                                                    nCharList.append(len(sentence))
                                    
                                    
# write to file
with open(filePath+'\\'+fileOut, 'w+', encoding="utf-8") as fOutput:
    fOutput.write("\n".join(sentencesFinal))
                                    
                                
                                #print(line_ind, sentence)
            


In [229]:
len(sentencesFinal)

8151

In [230]:
len(sentences)

22477

In [231]:
np.mean(nCharList)

96.28892160471108

In [232]:
np.max(nCharList)

272

In [233]:
np.min(nCharList)

21